In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.cm as cm

import matplotlib
matplotlib.rcParams.update({'font.size':18})
matplotlib.rcParams.update({'font.family':'serif'})

from matplotlib.colors import LogNorm

from astropy.io import fits

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# read the SNe data in
file = 'data/J_ApJ_716_712_tableb2.dat'


df = pd.read_table(file, delimiter='|', skiprows=7, header=None,
                   names=['SNe', 'S2N', 'Z', 'Bmag', 'Bmag_e', 'x1', 'x1_e', 'c', 'c_e', 'mu', 'mu_e', 'ref', 'fail'])

df

### Plot the redshift (Z) vs the distance modulus (mu)
This a classic figure for Supernova Cosmology!

(Put redshift on the X axis)

## BONUS
You could also convert $\mu$ to distance, using the formula:

$\mu = 5 \log_{10}(d) - 5$, 

where $d$ is in parsec.


You could also convert redshift ($z$) into velocity, using the approximation:

$v = c z$

where $c$ is the speed of light.

Now make the same diagram with units of Mpc (millions of pc) on the y-axis, and km/s on the X-axis

Fit a line to this data. What is the slope?

# Polynomial Fits

Let's try to fit the data using a simple polynomial fit function, built in to numpy...

**Experiment with different fit orders**

In [ ]:
ok = np.isfinite(df['mu']) # you'll want to make this cut to get rid of missing data!

DEG = 1 # this is the order of polynomial you want to fit
fit = np.polyfit(df['XAXIS'][ok], df['YAXIS'][ok], DEG)

plt.scatter(df['XAXIS'][ok], df['YAXIS'][ok]) # plot the data again (as above)

# now plot the FIT to the data...
plt.plot(df['XAXIS'][ok], np.polyval(fit, df['XAXIS'][ok]), 
         color='red', lw=3)



# Powerlaw Fit

It turns out we happen to know a bit about the physics of what we're looking at... so let's use that to make a smarter fit! In this case, the Y-axis (distance modulus) is in magnitudes, a logarithmic unit. Let's convert the X-axis to a log scaling and fit a straight line.

Note: this isn't *exactly* the same as fitting a true powerlaw to the data, but it's a good illustration. 
We're going to cheat a bit and fit the $\log_{10}(Z)$ space with a line


In [ ]:
fit2 = np.polyfit(np.log10(df['XAXIS'][ok]), df['YAXIS'][ok], 1)

plt.scatter(df['XAXIS'][ok], df['YAXIS'][ok]) # plot the data again (as above)

# now plot the FIT to the data...
plt.plot(df['XAXIS'][ok], np.polyval(fit2, np.log10(df['XAXIS'][ok])), 
         color='red', lw=3)



# Compare Fits

- Calculate the $\chi^2$ for the fits
    - $\chi^2 = 1/n$ $\Sigma$ ((data - model) / errors)$^2$
- Calculate the BIC
    - BIC = $\chi^2 + k$ $ln(n)$, where $k$ is the # of degrees of freedom in the model, $n$ the # of data points
    - https://en.wikipedia.org/wiki/Bayesian_information_criterion
- Which model is the "best"?
- the error on the distance modulus is the "mu_e" column of data

# Fit a Gaussian

Now let's branch out and fit another function. Today we'll keep it simple and just fit a simple Gaussian curve. BUT, you could do this approach to fit ANY function you can write down.

In [ ]:
# STEP 1: make a method that produces a function

def gaus(x, a, b, x0, sigma):
    """
    Simple Gaussian function

    Parameters
    ----------
    x : float or 1-d numpy array
        The data to evaluate the Gaussian over
    a : float
        the amplitude
    b : float
        the constant offset
    x0 : float
        the center of the Gaussian
    sigma : float
        the width of the Gaussian

    Returns
    -------
    Array or float of same type as input (x).
    """
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2)) + b


### This data is from the Sloan Digital Sky Survey. 

We're going to read in the spectrum of a Quasar and determine the redshift!

To do this, we'll fit an emission line with a gaussian to measure the wavelength center (x0 in the function above)

In [ ]:
# read the data in, just like last week...
dfile = 'data/spec-3819-55540-0186.fits'

hdulist = fits.open(dfile)
tbl = hdulist[1].data
hdr = hdulist[0].header
# tbl.columns
flux = tbl['flux']

### note the Flux is given, but NOT the wavelength... hmmm

In [ ]:
# SDSS spectra have many parameters in their "header" that define the properties of the spectrum.
# We'll use 2 of these to figure out the wavelength!
hdr

In [ ]:
# here is how you create the "log-linear" wavelength data using these header keywords
wave = 10. ** (np.arange(0,len(flux)) * hdr['COEFF1'] + hdr['COEFF0'])

# this may be useful to you someday!! Remember it

### Plot the spectrum...

In [ ]:
plt.figure(figsize=(11,5))
plt.plot( )
plt.xlim(5000,6000)

### Now fit the gaussian using SciPy's Curve_Fit method

There are 2 good tricks to doing this:

1. Give the function a reasonable initial guess (p0 below)
2. only fit the data with a small range of the emission line you care about

In [ ]:
p0 = (1, 2, 3, 4) # LOOK at the data above for this gaussian peak, put in good guesses!

# pick some limits within a few times the width of the peak, so to avoid (trying to) 
# fit the WHOLE spectrum with a single gaussian
WMIN = 4000
WMAX = 9000


x = np.where((wave > WMIN) & (wave < WMAX))
fit, cov = curve_fit(gaus, wave[x], flux[x], p0=p0)

plt.figure(figsize=(11,5))
plt.plot(wave, flux)
plt.plot(wave, gaus(wave, *fit)) # this *fit is a trick to explode all the parameters of "fit" in to "gauss"


# Compute the Redshift

Recall: 

$1 + z = \lambda_{obs} / \lambda_{rest}$

This is a Mg II line, with $\lambda_{rest} = 2800.3$ Angstroms 